In [4]:
from pylabrobot.liquid_handling import LiquidHandler
from pylabrobot.visualizer.visualizer import Visualizer
from pylabrobot.liquid_handling.backends.opentrons_backend_jump import OpentronsBackend
from pylabrobot.resources.opentrons import OTDeck
backend = OpentronsBackend(host="localhost", simulation=True)
lh = LiquidHandler(backend=backend, deck=OTDeck())
await lh.setup()
vis = Visualizer(resource=lh)
await vis.setup()

Websocket server started at http://127.0.0.1:2121
File server started at http://127.0.0.1:1337 . Open this URL in your browser.


Websocket server started at http://127.0.0.1:2122
File server started at http://127.0.0.1:1338 . Open this URL in your browser.


![The empty simulator](./img/visualizer/empty.png)

## Build the deck layout: Assigning plates and tips

When resources are assigned to the root resource of the Visualizer, in this case `lh`, they will automatically appear in the visualization.

In [3]:
from typing import cast

from pylabrobot.resources.tip_rack import TipRack


In [9]:
from pylabrobot.resources.opentrons import (
  opentrons_96_tiprack_300ul,
  nest_96_wellplate_2ml_deep
)

tip_rack = opentrons_96_tiprack_300ul(name='tip rack')
plt = nest_96_wellplate_2ml_deep(name='plate')

lh.deck.assign_child_at_slot(tip_rack, slot=7)
lh.deck.assign_child_at_slot(plt, slot=4)

![The simulator after the resources have been assigned](./img/visualizer/assignment.png)

### Configuring the state of the deck

As with every PyLabRobot script, you have the option of updating the state of the deck before you actually start your method. This will allow PyLabRobot to keep track of what is going on, enabling features like {func}`~pylabrobot.liquid_handling.liquid_handler.LiquidHandler.return_tips` and catching errors (like missed tips) before a command would be executed on the robot. With the visualizer, this state has the additional effect of updating the visualization.

### Tips

Let's use {func}`~pylabrobot.resources.tip_rack.fill` to place tips at all spots in the tip rack in location `0`.

In [10]:
tip_rack.fill()


You can precisely control the presence of tips using {func}`~pylabrobot.resources.tip_rack.set_tip_state`. This function allows you to set whether there is a tip in each {class}`~pylabrobot.resources.tip_rack.TipSpot`.

In [11]:
#tip_rack.set_tip_state(([True]*8 +[False]*8)*6)

### Liquids

Adding liquid to wells works similarly. You can use {func}`~pylabrobot.resources.plate.set_well_liquids` to set the liquid in each well of a plate. Each liquid is represented by a tuple where the first element corresponds to the type of liquid and the second to the volume in uL. Here, `None` is used to designate an unknown liquid.

In [12]:
plt_1_liquids = [[(None, 300)]]*96
plt.set_well_liquids(plt_1_liquids)

In the visualizer, you can see that the opacity of the well is proportional to how full the well is relative to its maximum volume.

![Simulator after the tips have been placed and the volumes adjusted](./img/visualizer/resources.png)

## Liquid handling

Once the layout is complete, you can run the same commands as described in the [basic liquid handling tutorial](../00_liquid-handling/hamilton-star/basic).

It is important that both tip tracking and volume tracking are enabled globally, so that the visualizer can keep track of the state of the tips and the volumes of the liquids.

In [13]:
from pylabrobot.resources import set_tip_tracking, set_volume_tracking
set_tip_tracking(True), set_volume_tracking(True)

(None, None)

### Picking up tips

Note that since we are using the {class}`~pylabrobot.liquid_handling.backends.chatterbox.ChatterboxBackend`, we just print out "Picking up tips" instead of actually performing an operation. The visualizer will show the tips being picked up.

In [14]:
print("left_pipette:", lh.backend.left_pipette)
print("right_pipette:", lh.backend.right_pipette)
print("left_has_tip:", lh.backend.left_pipette_has_tip)
print("right_has_tip:", lh.backend.right_pipette_has_tip)

left_pipette: {'pipetteId': 'sim_left', 'name': 'p300_single'}
right_pipette: None
left_has_tip: False
right_has_tip: False


In [15]:
await lh.pick_up_tips(tip_rack[0])

In [16]:
await lh.drop_tips(tip_rack[0])

### Aspirating and dispensing

In [17]:
await lh.pick_up_tips(tip_rack[1])

In [18]:
await lh.aspirate(plt["A2"], vols=[200])

In [19]:
await lh.dispense(plt["A1"], vols=[200])

In [20]:
await lh.return_tips()

In [21]:
await vis.stop()